In [ ]:
def what_is_installed():
    import pycaret
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except:
    !pip install prophet
    !pip install pycaret-ts-alpha
    what_is_installed()

In [ ]:
#%pip install --pre pycaret
#%pip install tsfresh
#%pip uninstall pycaret-ts-alpha

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import plotly.express as px
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests
from statsmodels.tsa.api import VAR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from numpy import log

In [ ]:
import os
import sys
import datetime as dt
#import missingno as mno 

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

LOAD DATA
--

In [ ]:
# Read the data into pandas DataFrames
may = pd.read_csv("MeasurementMay2022.csv")
june = pd.read_csv("Measurementjune2022(1).csv")
july = pd.read_csv("MeasurementsJuly2022.csv")
aug = pd.read_csv("MeasurementAugust2022.csv")


In [ ]:
may = may[may['data_date']!='01/06/2022']
june = june[june['data_date']!='01/07/2022']
july = july[july['data_date']!='01/08/2022']


In [ ]:
df= pd.concat([may,june,july,aug], axis=0)

In [ ]:
df.tail()

In [ ]:
df['date']=pd.to_datetime(df['data_date'] +" "+ df['data_time'],
               errors='raise',
                   utc="ns",
                   exact=True,
                   unit=None,
               format="%d/%m/%Y %H:%M:%S",
                   infer_datetime_format=True,
                   origin='unix', cache=True)

In [ ]:
df.tail()

In [ ]:
df.set_index("date", inplace=True)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
#rename columns with protacted name strings as well as the time and date column
df= df.rename(columns= {
                       "Magna_6 Segment 1_ (10-30cm) Soil Moisture_%":"segment1(10-30cm)",
                       "Magna_6 Segment 2_ (40-60 cm) Soil Moisture_%":"segment2(40-60cm)",
                       "Magna_6 Segment 3_ (70-90 cm) Soil Moisture_%":"segment3(70-90cm)",
                       "Magna_6 Segment 1_ (10-30 cm) Soil Electrical Conductivity_S/m":"segment1(EC)",
                       "Magna_6 Segment 2_ (40-60 cm) Soil Electrical Conductivity__S/m":"segment2(EC)",
                        "Magna_6 Segment 3_ (70-90 cm) Soil Electrical Conductivity__S/m":"segment3(EC)"
                       })
df.info()

In [ ]:
df.tail()

In [ ]:
df.index

In [ ]:
import plotly.graph_objects as go
fig1 = go.Figure(go.Scatter(x=df.index, y=df["segment1(10-30cm)"]))

fig1.show()


In [ ]:
df.info()

In [ ]:
### !!!!! CHECK!! If removing rows from the dataseet, doesnt this create a problem in the consistency of my timeline
## !!!!Should I instead replace it with mean of the previous readings ??????????

#19/04/2022	Installation on site
#22/05/2022	Problem with moisture probe – removed and cleaned
#22/05/2022	pH meter removed
#12/06/2022	New and old pH probe installed 
#06/07/2022	Testing pH first try
#27/07/2022	Testing pH second try
#13/08/2022	Changed batteries – data slave
#19/09/2022	Made changes to pH and conductivity sensor – old/new swap

##df[df.data_date != '19/05/2022']
## delete the above dates as required in the readme file provided

df.drop(df[df['data_date'] == '19/05/2022'].index, inplace=True)
df.drop(df[df['data_date'] == '22/05/2022'].index, inplace=True)

In [ ]:
## replace no-data with column mean
df['Metres above MSL'] = df['Metres above MSL'].replace(['no-data'],0.0)
df['TOW _MH2O'] = df['TOW _MH2O'].replace(['no-data'],0.0)

In [ ]:
df[df['Metres above MSL']== "no-data"]

In [ ]:
#convert string to float
df['Metres above MSL'] = pd.to_numeric(df['Metres above MSL'])#.astype(float)
df['TOW _MH2O'] = pd.to_numeric(df['TOW _MH2O'])#.astype(float)

In [ ]:
## check the dataframe

In [ ]:
del df['Temp_2']

del df['Temp_1']

del df['Temp_3']

df.info()

In [ ]:
#std of columns
std_dev=df.loc['22-05-21':'22-05-23'].std().sort_values(ascending=False)
std_dev

In [ ]:
variance=df.loc['22-05-21':'22-09-01'].var().sort_values(ascending=False)
variance

In [ ]:
#compare variance from all 3 segments


In [ ]:
segment1_var=df.loc['2022-06-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment1(10-30cm)'].var()
segment2_var=df.loc['2022-06-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment2(40-60cm)'].var()
segment3_var=df.loc['2022-06-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment3(70-90cm)'].var()
print("segment1: ", segment1_var )
print("segment2: ", segment2_var )
print("segment3: ", segment3_var )

In [ ]:
segment1_std=df.loc['2022-06-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment1(10-30cm)'].std()
segment2_std=df.loc['2022-06-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment2(40-60cm)'].std()
segment3_std=df.loc['2022-06-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment3(70-90cm)'].std()
print("segment1: ", segment1_std )
print("segment2: ", segment2_std )
print("segment3: ", segment3_std )

In [ ]:
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set(style="dark", color_codes=True)
#Compare the deviation for the last 4 months
plt.figure(figsize= (12,8))
sns.distplot(df.loc['2022-05-01 00:02:00+00:00':'2022-06-01 23:58:00+00:00']['segment1(10-30cm)'], 
            color= 'red',
            bins = 50)
plt.figure(figsize= (12,8))
sns.distplot(df.loc['2022-06-01 00:02:00+00:00':'2022-07-01 23:58:00+00:00']['segment1(10-30cm)'],
             color= 'green',
            bins = 50)
plt.figure(figsize= (12,8))
sns.distplot(df.loc['2022-07-01 00:02:00+00:00':'2022-08-01 23:58:00+00:00']['segment1(10-30cm)'],
             color= 'blue',
            bins = 50)
plt.figure(figsize= (12,8))
sns.distplot(df.loc['2022-08-01 00:02:00+00:00':'2022-09-01 23:58:00+00:00']['segment1(10-30cm)'],
             color= 'blue',
            bins = 50)

In [ ]:
# remove the date and time column
del df['data_date']

del df['data_time']

In [ ]:
df.plot(subplots=True, figsize=(12,65));

In [ ]:
#remove columns for segment 2 and 3
del df['segment2(40-60cm)']
del df['segment3(70-90cm)']
del df['segment2(EC)']
del df['segment3(EC)']

In [ ]:
#df.index
future_data = df.iloc[-7000:]
future_data

In [ ]:
# split data into train-test set
train = df[df.index < '2022-08-21']
test = df[df.index >= '2022-08-21']
# check shape
train.shape, test.shape


In [ ]:
from pycaret.time_series import TSForecastingExperiment

# Change renderer appropriately based on where the notebook is being run ----
# Refer to plotly for available renderers.
global_plot_settings = {"renderer": "colab",
                        "hoverinfo": "text"}


exp = TSForecastingExperiment()

target = 'segment1(10-30cm)'
data_for_modeling = df[df.index < '2022-08-21']
future_df = df[df.index >= '2022-08-21']
future_exog = future_df.drop(columns=target)
# init environment
exp.setup(data = df.reset_index(),
          target= target,
          seasonal_period=365,
          #fold_strategy = 'rolling',
          
          fig_kwargs=global_plot_settings,
          log_experiment=True,
          session_id=142)


In [ ]:
#.plot_model(plot="cv")
# Additional plots provided by PyCaret to help understand what is being done.
#plot_model(cv, plot="train_test_split")

In [ ]:
#Model training and selection
print("Model training and selection")
best_model = exp.compare_models()

In [ ]:
plot_model(best_model,
               plot = 'forecast')

In [ ]:
# Diagnostics plot
exp.plot_model(plot = 'diagnostics')

In [ ]:
tuned= exp.tune_model(best_model,fold=5)

In [ ]:
# forecast in unknown future
exp.plot_model(tuned,
               plot = 'forecast',
               data_kwargs = {'fh' : 1000})

In [ ]:
# Decomposition plot
exp.plot_model(tuned, plot = 'residuals')

In [ ]:
# Diagnostics plot
exp.plot_model(tuned, plot = 'diagnostics')

In [ ]:
# finalize model
final_best = exp.finalize_model(tuned)
print(final_best)

In [ ]:
# generate predictions
preds= exp.predict_model(final_best,fh=2000,X=future_exog)
print(preds)

In [ ]:
# save the model
exp.save_model(final_best, 'my_best_model')

In [ ]:
# NOTE: We will work on in-sample residuals that are provided by PyCaret automatically
# But if you need out of sample residuals, you can get them like this.
xresiduals = exp.get_config("y_test") - prediction
xresiduals

Feature Selection
## __

In [ ]:
#Building and displaying Correlation Matrix
corrMatrix = df.corr()
#print(corrMatrix)

_,ax=plt.subplots(1,1,figsize=(20,20))
sn.heatmap(df.corr(),annot=True, ax=ax,fmt='.2g', cbar=False)


In [ ]:
x = df.corr()
highly_corr = {}
for columns in x.columns:
  highly_corr[columns] = list(x[columns][x[columns]> 0.6].index)
#print(x['Air Pressure (x10)'][x['Air Pressure (x10)']> 0.5].index)

In [ ]:
highly_corr

In [ ]:
df.head()

In [ ]:
#Variance Threshold 

In [ ]:
from sklearn.feature_selection import VarianceThreshold 


#dfTreshold = df
# we are making a copy of dataframe and assigning to new variable 
# instead of pointing a new variable through the memory of the old file
dfTreshold = df.copy()

dfTreshold.info()

In [ ]:
VarThresh = VarianceThreshold(threshold = 0.8)
VarThresh.fit(dfTreshold)
VarThresh.get_support()

## All useless columns are False 

In [ ]:
# VIF -
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data = df.copy()
del vif_data['segment1(10-30cm)']
X = df[df.columns]
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
X

In [ ]:
## cut off treshhold
vif_data.sort_values(by='VIF', ascending = False)